In [ ]:
!pip install --upgrade google-cloud-storage
!pip install --upgrade pip
!pip install google.cloud.vision

In [0]:
import json
import requests
import plotly.plotly as py
import pandas as pd
import io
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credential.json"
# Imports the Google Cloud client library
from google.cloud import storage


Requirement already up-to-date: google-cloud-storage in /usr/local/lib/python3.6/dist-packages (1.14.0)
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.0.2)
  Using cached https://files.pythonhosted.org/packages/1c/61/b545aaee28d81e410e91d75e236a53f342bbb49fd4538332afa1661ef0ae/google_cloud_vision-0.35.2-py2.py3-none-any.whl


Getting image name from the User. This image would be uploaded by the user, and would be used to extract the details/features from the image.


In [0]:
dish_name="cauli.jpg"

Upload the content to the Google Cloud. Google Cloud vision is used to convert the image into features. 

In [0]:

# Instantiates a client
storage_client = storage.Client()

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))
    
upload_blob("bucketname",os.path.abspath(dish_name),"blob1")

File /content/cauli.jpg uploaded to blob1.


In [0]:
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

# Instantiates a client
client = vision.ImageAnnotatorClient()

# The name of the image file to annotate
file_name = os.path.abspath(dish_name)

# Loads the image into memory
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels:')
for label in labels:
    print(label.description)
  
buck = storage_client.get_bucket("bucketname")
blob_content=buck.blob('blob1')
blob_content.download_to_filename('temp.jpg')

# The name of the image file to annotate
file_name = os.path.abspath('temp.jpg')

# Loads the image into memory
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels:')
for label in labels:
    print(label.description)

Labels:
Food
Dish
Ingredient
Cuisine
Produce
Chicken meat
Recipe
Comfort food
Leaf vegetable
Labels:
Food
Dish
Ingredient
Cuisine
Produce
Chicken meat
Recipe
Comfort food
Leaf vegetable


Find the closest dish in Wegmans inventory.

In [0]:

from difflib import get_close_matches 
  
def closeMatches(patterns, word): 
     return (get_close_matches(word, patterns))   
  
  
#change response but keep key and api
response = requests.get("https://api.wegmans.io/meals/recipes?api-version=2018-10-18&Subscription-Key=")

print(response.status_code)

d = json.loads(response.text)
all_recipes = []


#change recipes to corresponding
for x in d['recipes']:
    all_recipes.append((x))

list_of_recipes = []
links_of_list_of_recipes = []
id_of_list_of_recipes = []

for i in range(len(all_recipes)):
  links_of_list_of_recipes.append(all_recipes[i]['_links'])
  id_of_list_of_recipes.append(all_recipes[i]['id'])
  list_of_recipes.append(all_recipes[i]['name'])

  
href_of_all_meals = []
for i in range(len(links_of_list_of_recipes)):
  href_of_all_meals.append(links_of_list_of_recipes[i][0]['href'])
  
  


200


In [0]:

search_query=labels[5].description





In [0]:

result = closeMatches(list_of_recipes , search_query) 

if(result):
  val = list_of_recipes.index(result[0])

search_item = 'https://api.wegmans.io/' + href_of_all_meals[val] + '&Subscription-Key='

response = requests.get(search_item)
print(response.status_code)
d = json.loads(response.text)


200


In [0]:
#Extract the contents from the API for every dish,and the dish from the image

# Make sure to add single quotes when entering your information to define them as strings
apikey = ''


In [0]:
# Print all information related to the Dish
id=str(d['id'])
url='https://api.wegmans.io/meals/recipes/'+id+'/?api-version=2018-10-18&subscription-key='+apikey
response = requests.get(url)
newid = json.loads(response.content)

In [0]:
ingredients=newid['ingredients']
print('Name of the Dish: '+str(newid['name']))
print('Wellness Ingredients: '+str(newid['wellnessKeys']))
url='https://api.wegmans.io'
print('Total ingredients required is '+str(len(ingredients)))
print('Names of all the Ingredients')
for i in range(len(ingredients)):
  link=ingredients[i].get('_links')
  print(ingredients[i]['name'])
  if(link!=None):
    #print(link[0].get('href'))  
    prod_name=ingredients[i]['_links'][0]['href']
    url_prod=url+prod_name+'&subscription-key='+apikey
    response = requests.get(url_prod)
    products = json.loads(response.content)
    df=pd.DataFrame.from_dict(products['nutrients'])
    print(df)
    #print(products['_links'][2]['href'])
    url_prices=url+products['_links'][2]['href']+'&subscription-key='+apikey
    response = requests.get(url_prices)
    prices = json.loads(response.content)
    print('Price: '+str(prices['stores'][0]['price']))
  



Name of the Dish: Chicken Parmesan
Wellness Ingredients: ['High Calcium']
Total ingredients required is 9
Names of all the Ingredients
Food You Feel Good About Seasoned Bread Crumbs
   dailyValuePercent dailyValueReference  \
0               0.00                None   
1               0.04                None   
2               0.00                None   
3               0.00                None   
4               0.21                None   
5               0.07                None   
6               0.02                None   

                           itemQuantity precision quantity                type  
0  {'value': 10, 'unitOfMeasure': 'OZ'}      None     0 mg         Cholesterol  
1  {'value': 10, 'unitOfMeasure': 'OZ'}      None      1 g       Dietary Fiber  
2  {'value': 10, 'unitOfMeasure': 'OZ'}      None     30 g   MetricServingSize  
3  {'value': 10, 'unitOfMeasure': 'OZ'}      None      0 g       Saturated Fat  
4  {'value': 10, 'unitOfMeasure': 'OZ'}      None   500 mg  

In [0]:
#Extract the store information for the nearest location

response = requests.get("https://api.wegmans.io/stores/?api-version=2018-10-18&Subscription-Key=")

print(response.status_code)

d = json.loads(response.text)
d1=[]

#change recipes to corresponding
for x in d['stores']:
    d1.append(x)
#     print(x)

d2=d['stores']

200


In [0]:
links_of_stores = []
latitudes = []
longitudes = []
shopname = []
shoplocation = []


for i in range(len(d2)):
  links_of_stores.append(d2[i]['_links'])
  latitudes.append(int(d2[i]['latitude']))
  longitudes.append(int(d2[i]['longitude']))
  shopname.append(d2[i]['name'])
  shoplocation.append(d2[i]['number'])

  
href_of_all_stores = []
for i in range(len(links_of_stores)):
  href_of_all_stores.append(links_of_stores[i][0]['href'])
  

  

In [0]:
#!pip install geopy

from geopy.distance import geodesic
import geopy.distance

def find_shortest_store(lat, long, userlat, userlong): 
  # approximate radius of earth in km
  mini = []
  print(len(lat))
  for i in range(len(lat)):
    coords_1 = (lat[i], long[i])
    coords_2 = (userlat, userlong)
    mini.append(geodesic(coords_1, coords_2).miles)
  return mini

In [0]:
response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=Montrose+Avenue,+Buffalo,+NY &key=')

resp_json_payload = response.json()

userlat = (resp_json_payload['results'][0]['geometry']['location']['lat'])
userlong = (resp_json_payload['results'][0]['geometry']['location']['lng'])

#resp_json_payload

shortest_path = find_shortest_store(latitudes, longitudes, userlat, userlong)

a =  shortest_path.index(min(shortest_path))

min(shortest_path)



111


42.107275836852345

In [0]:
print(shopname[a] + ' is the shortest shop near the customers house')



print("The url is " + str(links_of_stores[a][0]))

NIAGARA FALLS BLVD. is the shortest shop near the customers house
The url is {'href': '/stores/86?api-version=2018-10-18', 'rel': 'self', 'type': 'GET'}
